In [3]:
import boto3
import json
import pprint
import time
import uuid
import urllib

bucket = str(uuid.uuid1())

print ('bucket:', bucket)

region = 'ap-northeast-1'

s3 = boto3.client('s3', region)

result = s3.create_bucket(
    Bucket=bucket,
    CreateBucketConfiguration={'LocationConstraint' : region}
)

bucket: 6bfb9d1c-5f06-11ed-a899-802bf98abdc6


In [4]:
file = 'scribe_file_in.mp3'
key = 'input'
s3.upload_file(file, bucket, key)

In [13]:
transcribe = boto3.client('transcribe', region)
job = str(uuid.uuid1())
uri = 'https://s3-' + region + '.amazonaws.com/' + bucket + '/' + key
result = transcribe.start_transcription_job(
TranscriptionJobName=job, Media={'MediaFileUri' : uri},
MediaFormat='mp3', LanguageCode='en-US')
print ('start_transcription_job:')
pprint.pprint(result)

start = time.time()
while True:
    result = transcribe.get_transcription_job(TranscriptionJobName=job)
    status = result['TranscriptionJob']['TranscriptionJobStatus']
    if status != 'IN_PROGRESS' :
        break
    time. sleep(10)
    print('time:', time.time()-start)
print('get_transcription_job:')
pprint.pprint(result)

BadRequestException: An error occurred (BadRequestException) when calling the StartTranscriptionJob operation: The URI that you provided doesn't point to an S3 object. Make sure that the object exists and try your request again.

In [11]:
uri = result['TranscriptionJob']['Transcript']['TranscriptFileUri']
print('uri:', uri)
with urllib.request.urlopen(uri) as file_in:
    transcripts = json.load(file_in)

with open('scribe_file_out.json', 'w', encoding='utf-8') as file_out:
    json.dump(transcripts, file_out, indent=4)
    
print ( 'transcript:')
for transcript in transcripts['results']['transcripts']:
    print(transcript['transcript'])

uri: https://s3.ap-northeast-1.amazonaws.com/aws-transcribe-ap-northeast-1-prod/552947388436/71da9719-5f09-11ed-99c1-802bf98abdc6/ddc7c348-3289-47f3-b78e-e096bd1cd551/asrOutput.json?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEEEaDmFwLW5vcnRoZWFzdC0xIkcwRQIgJclKBejI8FaQ%2FFFhMvRYzuDJDXTP90m3Le6F8gfDq2cCIQDNhaYYwDoNT90nDhYuJBscjitTLyIZo2POXAk9hYw2kSrcBAg6EAMaDDAxNTUzMzUyNzE5NSIMpOXVVOgbuCjKA5jdKrkEegC%2BDAgQ4kiPZi1m5HazFNbHicDT3cCh%2FkcUUXfnqzUd6qlRZKgwKdbFC3MOOtRzbsi4hnV131fsapBbRw8voP3X8Zc6mK6G%2F1ccH30RWkKi%2BdTFWP4xHUm2yHmJe0c8Bn0YtWtbeEj5qdWd0ElyH%2BzkQJeOKo89JGeMxi1tefAZQTZMo7EXzPtDtBC5Er8zTodFzXeq0eiN5yo3%2FkD3%2BTfMAZO54rIKp8iQnKPlfyC7OK%2Bt6m4U97hnVP2ChJ%2Byt4ytbmXp7d0bZE6vnk%2BEYo687Khy1ZlXhJn%2F7RbS0XOQy%2BfuuoxZPBIu%2FjMW8%2Fc%2BxQbrOVmEYMx51OH84W3cYB8ivASFtpC%2BAMpnh89e6QCHxYasbPrTsZHWZHBJiihp%2BlI50a%2BkY%2FjGRMtGupUyzQ5t6lW5Tajg3Q2XRrOis%2FXem4xu6S0eD5bd5DZTNAD5HFkpMpsW0vEBqSlxw8V5VSiLGH1uqUQx5lJJrVxt%2FBl1R7ICdj8kEp9ur3BCULXmu4N%2BQaW7az%2FxPC3w0gf5MjDLxcja2Gps5Z04XrfdbXYP3MLg7

In [12]:
transcribe.delete_transcription_job (TranscriptionJobName=job)
s3.delete_object(Bucket=bucket, Key=key)
s3.delete_bucket(Bucket=bucket)

{'ResponseMetadata': {'RequestId': 'ZE8WJ4VJMT2KPCW3',
  'HostId': 'ULW8+XyEPc0hMlAsHFY8dJSa/Fcd1rCQXcPwgEMzgQISqZYlbqBcwy+1Axu5ekIKyxiI2E5N6PA=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'ULW8+XyEPc0hMlAsHFY8dJSa/Fcd1rCQXcPwgEMzgQISqZYlbqBcwy+1Axu5ekIKyxiI2E5N6PA=',
   'x-amz-request-id': 'ZE8WJ4VJMT2KPCW3',
   'date': 'Tue, 08 Nov 2022 02:06:49 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}